In [1]:
import numpy as np
import pandas as pd
import itertools


from konlpy.tag import Okt
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer

### 전처리가 이미 완료된 후의 데이터를 사용
보안상 데이터를 전체 공개 불가

In [2]:
df = pd.read_csv('../data/dataset/CAI_data_003.csv')

In [3]:
df['sentence'][1]

'고객 요청사항 수신불인터넷 연결오류 뜸재부팅해 봄 모텔 영업용 일중 꼭 점검원하여 지사 이관함회선 단말특이사항 진단 결과 기타 추가 연락처'

In [4]:
okt = Okt()

tokenized_doc = okt.pos(df['sentence'][1])
tokenized_nouns = ' '.join([word[0] for word in tokenized_doc if word[1] != 0])

print('품사 태깅 10개만 출력 :',tokenized_doc[:10])
print('추출 :',tokenized_nouns)

품사 태깅 10개만 출력 : [('고객', 'Noun'), ('요청', 'Noun'), ('사항', 'Noun'), ('수', 'Modifier'), ('신', 'Modifier'), ('불', 'Noun'), ('인터넷', 'Noun'), ('연결', 'Noun'), ('오류', 'Noun'), ('뜸', 'Noun')]
추출 : 고객 요청 사항 수 신 불 인터넷 연결 오류 뜸 재부팅 해 봄 모텔 영업 용 일 중 꼭 점검 원하여 지사 이 관함 회선 단말 특이 사항 진단 결과 기타 추가 연락처


In [5]:
n_gram_range = (2, 3)

count = CountVectorizer(ngram_range=n_gram_range).fit([tokenized_nouns])
candidates = count.get_feature_names_out()

print('trigram 개수 :',len(candidates))
print('trigram 다섯개만 출력 :',candidates[:5])

trigram 개수 : 41
trigram 다섯개만 출력 : ['결과 기타' '결과 기타 추가' '고객 요청' '고객 요청 사항' '관함 회선']


In [6]:
model = SentenceTransformer('sentence-transformers/xlm-r-100langs-bert-base-nli-stsb-mean-tokens')
doc_embedding = model.encode([df['sentence'][1]])
candidate_embeddings = model.encode(candidates)

In [7]:
top_n = 5
distances = cosine_similarity(doc_embedding, candidate_embeddings)
keywords = [candidates[index] for index in distances.argsort()[0][-top_n:]]
print(keywords)

['점검 원하여', '연결 오류 재부팅', '원하여 지사 관함', '점검 원하여 지사', '영업 점검 원하여']


In [8]:
def max_sum_sim(doc_embedding, candidate_embeddings, words, top_n, nr_candidates):
    # 문서와 각 키워드들 간의 유사도
    distances = cosine_similarity(doc_embedding, candidate_embeddings)

    # 각 키워드들 간의 유사도
    distances_candidates = cosine_similarity(candidate_embeddings,
                                            candidate_embeddings)

    # 코사인 유사도에 기반하여 키워드들 중 상위 top_n개의 단어를 pick.
    words_idx = list(distances.argsort()[0][-nr_candidates:])
    words_vals = [candidates[index] for index in words_idx]
    distances_candidates = distances_candidates[np.ix_(words_idx, words_idx)]

    # 각 키워드들 중에서 가장 덜 유사한 키워드들간의 조합을 계산
    min_sim = np.inf
    candidate = None

    for combination in itertools.combinations(range(len(words_idx)), top_n):
        sim = sum([distances_candidates[i][j] for i in combination for j in combination if i != j])
        if sim < min_sim:
            candidate = combination
            min_sim = sim

    return [words_vals[idx] for idx in candidate]

def mmr(doc_embedding, candidate_embeddings, words, top_n, diversity):

    # 문서와 각 키워드들 간의 유사도가 적혀있는 리스트
    word_doc_similarity = cosine_similarity(candidate_embeddings, doc_embedding)

    # 각 키워드들 간의 유사도
    word_similarity = cosine_similarity(candidate_embeddings)

    # 문서와 가장 높은 유사도를 가진 키워드의 인덱스를 추출.
    # 만약, 2번 문서가 가장 유사도가 높았다면
    # keywords_idx = [2]
    keywords_idx = [np.argmax(word_doc_similarity)]

    # 가장 높은 유사도를 가진 키워드의 인덱스를 제외한 문서의 인덱스들
    # 만약, 2번 문서가 가장 유사도가 높았다면
    # ==> candidates_idx = [0, 1, 3, 4, 5, 6, 7, 8, 9, 10 ... 중략 ...]
    candidates_idx = [i for i in range(len(words)) if i != keywords_idx[0]]

    # 최고의 키워드는 이미 추출했으므로 top_n-1번만큼 아래를 반복.
    # ex) top_n = 5라면, 아래의 loop는 4번 반복됨.
    for _ in range(top_n - 1):
        candidate_similarities = word_doc_similarity[candidates_idx, :]
        target_similarities = np.max(word_similarity[candidates_idx][:, keywords_idx], axis=1)

        # MMR을 계산
        mmr = (1-diversity) * candidate_similarities - diversity * target_similarities.reshape(-1, 1)
        mmr_idx = candidates_idx[np.argmax(mmr)]

        # keywords & candidates를 업데이트
        keywords_idx.append(mmr_idx)
        candidates_idx.remove(mmr_idx)

    return [words[idx] for idx in keywords_idx]

In [9]:
sentences = []

stopwords = ['고객','요청사','항','요청사항','안심권유','기타 추가 연락처']

for sentence in df['sentence']:
    for word in stopwords:
        sentence = sentence.replace(word, '')
        sentence = sentence.lstrip()
    sentences.append(sentence)
df['sen'] = sentences

In [10]:
df['sen']

0                                            안됨상담 중 자연회복됨
1       수신불인터넷 연결오류 뜸재부팅해 봄 모텔 영업용 일중 꼭 점검원하여 지사 이관함회선...
2                                  수신불회선 단말특이사 진단 결과  연후방
3                                       수신 불네트워크 오류 접수 보류
4                              수신불 연락 후회선 단말특이사 진단 결과 인 입
                              ...                        
8661    화질 불량 시청 중 화면 멈추고 꺼졌다 켜지는 현상이 있다고 함 연락 후 방문요회선...
8662                       리콜넷플릭스 연령인증 문의 건 리콜하니 해결했다고 하심
8663                                           대수신불전송 후리셋
8664    빠른 점검 요청   시청 불인터넷 연결오류나 옴 모든 장비 재부팅 안내회선 단말특이...
8665      수신불 연락 후 방문지니 셋 탑 전원안 들어옴 교체 요청회 선 단말특이사 진단 결과 
Name: sen, Length: 8666, dtype: object

In [ ]:
abstract_sen = []


for i in range(len(df)) :
    

    try :
        okt = Okt()

        tokenized_doc = okt.pos(df['sen'][i])
        tokenized_nouns = ' '.join([word[0] for word in tokenized_doc if word[1] != 0])

        n_gram_range = (2, 3)

        count = CountVectorizer(ngram_range=n_gram_range).fit([tokenized_nouns])
        candidates = count.get_feature_names_out()

        doc_embedding = model.encode([df['sen'][i]])
        candidate_embeddings = model.encode(candidates)

        distances = cosine_similarity(doc_embedding, candidate_embeddings)
        keywords = [candidates[index] for index in distances.argsort()[0][-top_n:]]
        
        
        keywords = max_sum_sim(doc_embedding, candidate_embeddings, candidates, top_n=3, nr_candidates=10)
        
        
        abstract_sen.append(keywords)
        print('----------------------------------------------------------')
        print('원본 : ', df['sen'][i])
        print('요약 : ', abstract_sen[i])
        print('----------------------------------------------------------')

    except :
        abstract_sen.append(df['sen'][i])

----------------------------------------------------------
원본 :  안됨상담 중 자연회복됨
요약 :  ['자연 회복', '상담 자연', '안됨 상담']
----------------------------------------------------------
----------------------------------------------------------
원본 :  수신불인터넷 연결오류 뜸재부팅해 봄 모텔 영업용 일중 꼭 점검원하여 지사 이관함회선 단말특이사 진단 결과 
요약 :  ['오류 재부팅 모텔', '점검 원하여', '모텔 영업 점검']
----------------------------------------------------------
----------------------------------------------------------
원본 :  수신불회선 단말특이사 진단 결과  연후방
요약 :  ['특이 진단 결과', '결과 후방', '회선 단말']
----------------------------------------------------------
----------------------------------------------------------
원본 :  수신 불네트워크 오류 접수 보류
요약 :  ['네트워크 오류', '수신 네트워크', '접수 보류']
----------------------------------------------------------
----------------------------------------------------------
원본 :  수신불 연락 후회선 단말특이사 진단 결과 인 입
요약 :  ['특이 진단 결과', '회선 단말', '단말 특이']
----------------------------------------------------------
---------------------------------------------------

----------------------------------------------------------
원본 :  수신 불방쪽만 안됨 셋탑재 부팅 후 정상
요약 :  ['안됨 탑재', '수신 불방쪽', '탑재 부팅 정상']
----------------------------------------------------------
----------------------------------------------------------
원본 :  수신불인터넷 연결오류 뜸재부팅해 봄 연락 먼저 회선 단말특이사 진단 결과 
요약 :  ['진단 결과', '연결 오류 재부팅', '먼저 회선']
----------------------------------------------------------
----------------------------------------------------------
원본 :  가능한 일방 요 분 정도 시청되면 화면 꺼짐 소리 정상회선 단말특이사 진단 결과  코로나 미 대상
요약 :  ['진단 결과 코로나', '정도 시청 되면', '화면 꺼짐 소리']
----------------------------------------------------------
----------------------------------------------------------
원본 :  일반 아들 방안됨 전원 적색점 등재부팅해도 동일 기량 자 방문 요청함 방문 시간 참고
요약 :  ['일반 아들 방안', '동일 기량 방문', '기량 방문 요청']
----------------------------------------------------------
----------------------------------------------------------
원본 :  안됨 셋 탑 전원 빨간불껏 다 켜도 아무 반응 없음 최대한 빠른 방문 요청
요약 :  ['최대한 빠른 방문', '없음 최대한', '켜도 아무 반응']
------------------------

----------------------------------------------------------
원본 :  수신불 연락 후 전원회선 단말특이사 진단 결과 인 입
요약 :  ['특이 진단', '전원 회선 단말', '연락 전원']
----------------------------------------------------------
----------------------------------------------------------
원본 :  리모컨 작동 불리모컨에 버튼을 안 눌러도 리모컨에 불이 계속 들어오면서 오작동이 됨연 후방 회선 단말특이사 진단 결과 
요약 :  ['리모컨 계속 들어오면서', '작동 후방 회선', '리모컨 버튼']
----------------------------------------------------------
----------------------------------------------------------
원본 :  수신불인터넷 오류회선 단말특이사 진단 결과  연후방
요약 :  ['진단 결과', '오류 회선', '회선 단말']
----------------------------------------------------------
----------------------------------------------------------
원본 :  수신 불네트워크 오류 홈 너무 뜨겁다며 문제 있는 건 아닌 지 방문해서 확인 요회선 단말특이사 진단 결과 
요약 :  ['있는 아닌 방문', '해서 확인', '오류 너무 뜨겁다며']
----------------------------------------------------------
----------------------------------------------------------
원본 :  리모컨 안됨 가까이에서는 되고 멀리서는 안됨 리모컨 페어링재부팅에 택배 접수함
요약 :  ['서는 안됨 리모컨', '재부팅 택배 접수', '택배 접수']
---------

----------------------------------------------------------
원본 :  자연회복회선 단말특이사 진단 결과 
요약 :  ['특이 진단', '회선 단말', '자연 회복']
----------------------------------------------------------
----------------------------------------------------------
원본 :  장비 장애 안내
요약 :  ['장애 안내', '장비 장애', '장비 장애 안내']
----------------------------------------------------------
----------------------------------------------------------
원본 :  화질이 상장비리셋 안내 후 정상회선 단말특이사 진단 결과 정상기 타 추가 연락처
요약 :  ['안내 정상 회선', '추가 연락처', '화질 장비 리셋']
----------------------------------------------------------
----------------------------------------------------------
원본 :  셋탑박스 전원선을 뺐다 꼽아야 전원이 켜진다고 함 연락 후 방문회선 단말 특이사 진단 결과 
요약 :  ['꼽아 전원', '셋탑박스 원선 뺐다', '꼽아 전원 켜진다고']
----------------------------------------------------------
----------------------------------------------------------
원본 :  티비끊김회선 단말특이사 진단 결과 
요약 :  ['티비 끊김 회선', '특이 진단 결과', '회선 단말']
----------------------------------------------------------
--------------------------------------

----------------------------------------------------------
원본 :  리셋 시청 불수신 미약회선 단말특이사 진단 결과 
요약 :  ['회선 단말', '미약 회선 단말', '시청 불수신 미약']
----------------------------------------------------------
----------------------------------------------------------
원본 :  수신 불셋탑리셋동일회선단말특이사 진단 결과 
요약 :  ['단말 특이 진단', '수신 불셋탑', '리셋 동일 회선']
----------------------------------------------------------
----------------------------------------------------------
원본 :  리모컨지참까지 해서 방문 요청함 추가일 오전 시시 방문 안내함수 신불 화면 안나 옴셋탑재 부팅해도 동일 증상 언급
요약 :  ['방문 안내 함수', '부팅 해도 동일', '참까지 해서']
----------------------------------------------------------
----------------------------------------------------------
원본 :  아트빌 수신불시 방문 요청 외부 회선 단말특이사 임의 코로나 이상무진단 결과 
요약 :  ['코로나 이상무', '수신 불시 방문', '요청 외부']
----------------------------------------------------------
----------------------------------------------------------
원본 :  리모컨 잘못 건드리고 수신 불회선 단말특이사 진단 결과  엔지니어 확인되면 연락 먼저 요청
요약 :  ['건드리고 수신 회선', '리모컨 잘못 건드리고', '되면 연락 먼저']
-----------------

...

----------------------------------------------------------
원본 :  다시보기 안됨 셋 탑 전 원재연결회선 단말특이사 진단 결과 코로나 이상무
요약 :  ['보기 안됨', '결과 코로나 이상무', '회선 단말 특이']
----------------------------------------------------------
----------------------------------------------------------
원본 :  채널 정보표시 안됨연 후방 회선 단말특이사 진단 결과 
요약 :  ['안됨 후방', '정보 표시', '회선 단말']
----------------------------------------------------------
----------------------------------------------------------
원본 :  차회무 응 접수 보류차리셋 접수 수신불
요약 :  ['보류 리셋', '보류 리셋 접수', '접수 보류']
----------------------------------------------------------
----------------------------------------------------------
원본 :  코로나 이 상무재확인 필   스마트가 고장이 나서 아나로그를 교체할려고 함 케이블 교체 요청회선 단말특이사 진단 결과 인 입
요약 :  ['나서 나로', '스마트 고장 나서', '교체 할려고']
----------------------------------------------------------
----------------------------------------------------------
원본 :  리셋 접수  티수신불회선 단말특이사 포트리셋진단 결과 정상기 타 추가 연락처
요약 :  ['접수 회선', '정상기 추가 연락처', '특이 포트 리셋']
--------------------------------------

----------------------------------------------------------
원본 :  코로나 이 상무재확인 필   수신 불신가 약하다고 함셋탑 노후화로 인해 셋탑 교체 요청회 선 단말특이사 진단 결과 정상기 타 추가 연락처
요약 :  ['정상기 추가 연락처', '함셋탑 노후 화로', '수신 불신 약하다고']
----------------------------------------------------------
----------------------------------------------------------
원본 :  대수신불회선 단말특이사 진단 결과 
요약 :  ['특이 진단 결과', '회선 단말', '단말 특이']
----------------------------------------------------------
----------------------------------------------------------
원본 :  코로나 이 상무재확인 필티 수신불회선 단말특이사 진단 결과 
요약 :  ['회선 단말 특이', '코로나 상무', '확인 필티']
----------------------------------------------------------
----------------------------------------------------------
원본 :  최대한 빠른 방문요일 오후   거실 리모컨 채널 변경 안됨회선 단말특이사 진단 결과 인
요약 :  ['채널 변경 안됨', '방문 요일', '오후 거실 리모컨']
----------------------------------------------------------
----------------------------------------------------------
원본 :  안방수신불회선 단말특이사 진단 결과 
요약 :  ['특이 진단 결과', '진단 결과', '안방 회선 단말']
----------------------------------

----------------------------------------------------------
원본 :  리셋 접수   전체 리셋해도 증상 동일시청 불엔지니어 방문 접수 거부로 보류회선 단말특이사 진단 결과 인
요약 :  ['전체 리셋', '해도 증상 동일', '리셋 해도 증상']
----------------------------------------------------------
----------------------------------------------------------
원본 :  화면이 꺼졌다 나왔다 함 잦은 고장 최대한 꼼꼼하게 점검 요청회선 단말특이사 진단 결과 
요약 :  ['점검 요청 회선', '꼼꼼하게 점검', '고장 최대한 꼼꼼하게']
----------------------------------------------------------
----------------------------------------------------------
원본 :  수신 불량 확인 불가회선 단말특이사 코로나무 진단 결과 
요약 :  ['회선 단말 특이', '코로 나무 진단', '불량 확인 불가']
----------------------------------------------------------
----------------------------------------------------------
원본 :  고양대리접수 요청 건   수신불회선 단말특이사 진단 결과 
요약 :  ['회선 단말', '접수 요청', '고양 대리 접수']
----------------------------------------------------------
----------------------------------------------------------
원본 :  기가 지니 음성인식이상셋탑 윗면 빨간불 들어와 있음 방문 전 연락 요회선 단말특이사 진단 결과 
요약 :  ['연락 회선 단말', '있음 방문 연락', '음성인식 이상 셋탑']
-

----------------------------------------------------------
원본 :  연 후방 코로나 이 상무 권유 실패   시청 불회선 단말특이사 진단 결과 
요약 :  ['특이 진단', '시청 회선', '실패 시청']
----------------------------------------------------------
----------------------------------------------------------
원본 :  수신 불공사 장애로 확인되어서 안내 후 수긍공사 완료되는 대로 연락갈 수 있도록 하겠다고 안내 함회선 단말특이사 진단 결과 정상기 타 추가 연락처 코로나 안내유료화 안내ㄴ 님 연락처
요약 :  ['연락 있도록', '유료 안내 연락처', '대로 연락 있도록']
----------------------------------------------------------
----------------------------------------------------------
원본 :  일리셋 수신불 업데이트 오류뜸리셋
요약 :  ['오류 리셋', '업데이트 오류', '업데이트 오류 리셋']
----------------------------------------------------------
----------------------------------------------------------
원본 :  코로나 이 상무재확인 필   리모컨 고장으로 교체 요청회 선 단말특이사 진단 결과 
요약 :  ['상무 확인', '으로 교체 요청', '단말 특이']
----------------------------------------------------------
----------------------------------------------------------
원본 :  이선희 서비스센터 접수 요청 건   사용 불외부 신고회선 단말특이사 진단 결과 
요약 :  ['이선희 서비스 센터', '접수 요청',

----------------------------------------------------------
원본 :  코로나 이 상무재확인 필티 수신 불잦은 고장으로 불만회선 단말특이사 진단 결과 
요약 :  ['상무 확인 필티', '필티 수신', '불만 회선 단말']
----------------------------------------------------------
----------------------------------------------------------
원본 :  월일 시작동이상 올인원 정상코로나 이 상무교체 요청 센터 대리신고
요약 :  ['코로나 상무', '이상 올인원 정상', '요청 센터 대리']
----------------------------------------------------------
----------------------------------------------------------
원본 :  리모컨 작동불교체 요청회 선 단말특이사 진단 결과 정상기 타 추가 연락처
요약 :  ['결과 정상기 추가', '리모컨 교체 요청', '요청 단말']
----------------------------------------------------------
----------------------------------------------------------
원본 :  수신 불상담 중 자연회복업 성공회 선단말특이사 진단 결과 
요약 :  ['특이 진단 결과', '수신 상담 자연', '성공회 선단']
----------------------------------------------------------
----------------------------------------------------------
원본 :  수신불회선 단말특이사 진단 결과  연후방
요약 :  ['특이 진단 결과', '결과 후방', '회선 단말']
----------------------------------------------------------

In [ ]:
df['sentence']

In [ ]:
abstract_sen

## Max Sum Similarity
데이터 쌍 사이의 최대 합 거리는 데이터 쌍 간의 거리가 최대화되는 데이터 쌍으로 정의됩니다. 여기서의 의도는 후보 간의 유사성을 최소화하면서 문서와의 후보 유사성을 극대화하고자 하는 것입니다.